In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
import collections
import helper
import numpy as np
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from keras import models, layers
from keras import backend
import pickle
from tensorflow.keras.layers import Bidirectional, Conv1D, MaxPool1D, Dense, Flatten, Dropout, AveragePooling2D, LSTM, TimeDistributed, Attention
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score,recall_score,precision_score, confusion_matrix
from keras_self_attention import SeqSelfAttention

In [ ]:
pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Fifth Competition/

%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Fifth Competition
Anything-v3.ipynb  Neural_Machine_Translation.ipynb  train-05/
models/            test-05/


In [ ]:
def read_data(file_name):
    output = []
    cur_sent = []
    source_file = open(file_name, 'r', encoding='utf-8')
    text = source_file.read()

    for current in (text.split('\n')):
        if current == '<s>':
            cur_sent = []
            continue
        if current == '</s>':
            output.append(cur_sent)
            continue
        if current in '()':
            continue
        else:
            cur_sent.append(current.lower())

    return output

In [ ]:
%ls

Anything-v3.ipynb  Neural_Machine_Translation.ipynb  train-05/
models/            test-05/


In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

In [ ]:
source_train = read_data('train-05/train-source.txt')
target_train = read_data('train-05/train-target.txt')

# source = read_data('test-05/test-source.txt')
# target = read_data('test-05/test-target.txt')

source = read_data('train-05/train-source.txt')
target = read_data('train-05/train-target.txt')


len(source)

45171

In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

preproc_source_train, preproc_target_train, source_train_tokenizer, target_train_tokenizer =\
    preprocess(source_train, target_train)
    
max_source_train_length = preproc_source_train.shape[1]
max_target_train_length = preproc_target_train.shape[1]
source_train_vocab_size = len(source_train_tokenizer.word_index)
target_train_vocab_size = len(target_train_tokenizer.word_index)

max_train_target_length = preproc_target_train.shape[1]

print('Data Preprocessed')
print("Max source sentence length:", max_source_train_length)
print("Max target sentence length:", max_target_train_length)
print("Source vocabulary size:", source_train_vocab_size)
print("Target vocabulary size:", target_train_vocab_size)

Data Preprocessed
Max source sentence length: 231
Max target sentence length: 221
Source vocabulary size: 31035
Target vocabulary size: 25735


In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

preproc_source, preproc_target, source_tokenizer, target_tokenizer =\
    preprocess(source, target)
    
max_source_length = preproc_source.shape[1]
max_target_length = preproc_target.shape[1]
source_vocab_size = len(source_tokenizer.word_index)
target_vocab_size = len(target_tokenizer.word_index)

max_train_target_length = preproc_target.shape[1]

print('Data Preprocessed')
print("Max source sentence length:", max_source_length)
print("Max target sentence length:", max_target_length)
print("Source vocabulary size:", source_vocab_size)
print("Target vocabulary size:", target_vocab_size)

Data Preprocessed
Max source sentence length: 231
Max target sentence length: 221
Source vocabulary size: 31035
Target vocabulary size: 25735


In [ ]:
max_target_train_length

221

In [ ]:
#########
tmp_x = pad(preproc_source, max_target_train_length)
tmp_x = tmp_x.reshape((-1, preproc_target_train.shape[-2], 1))

In [ ]:
# # source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
# # Train bi-directional
# tmp_x = pad(preproc_source, preproc_target.shape[1])
# tmp_x = tmp_x.reshape((-1, preproc_target.shape[-2], 1))

# bidi_model = bd_model(
#     tmp_x.shape,
#     preproc_target.shape[1],
#     len(source_tokenizer.word_index)+1,
#     len(target_tokenizer.word_index)+1)


# bidi_model.fit(tmp_x, preproc_target, batch_size=32, epochs=5, validation_split=0.2)

# # Print prediction(s)
# print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
# bidi_model.save('./models/bidire_5epoch_model.h5')

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

In [ ]:
# Train Simple RNN
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_target_length,
    source_vocab_size,
    target_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_target, batch_size=32, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
bidi_model.save('./models/new_rnn_model.h5')

In [ ]:
from keras.models import load_model

# loaded_bid = load_model("./models/bidire_model.h5")

In [ ]:
predictions = loaded_bid.predict(tmp_x[:8])[0]

In [ ]:
real = (tmp_x[:4])[0]
real2 = logits_to_text(real, target_tokenizer)
real2

In [ ]:
final_words = logits_to_text(predictions, target_tokenizer)
(final_words)

In [ ]:
# simple_rnn_model.save('attention')
# print(logits_to_text(loaded_bid.predict(tmp_x[:1])[0]),target_tokenizer)
predictions = loaded_bid.predict(tmp_x[:1])[0]
one = tmp_x[0]

In [ ]:
index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0]